In [1]:
import os
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
HTML("<style>.rendered_html th {max-width: 120px;}</style>")

In [142]:
def readTerminology(file):
    "Reads a terminology file into two dictionaries."
    cuiName = {}
    nameCui = {}
    with open(file) as f:
        for m in f.readlines():
            s = m.split('||')
            cui = s[0]
            names = s[1].strip().split('|')
            cuiName[cui] = names

            for name in names:
                if name not in nameCui:
                    nameCui[name] = []
                nameCui[name].append(cui)
    return cuiName, nameCui

def readAnnotations(path):
    "Reads all .concept files from path into single dataframe."
    annotations = pd.DataFrame([])
    for file in os.listdir(path):
        if '.concept' in file:
            df = pd.read_table(f'{path}/{file}',sep='\|\|',header=None, names=['file_id','ix','type','name','cui'])
            df['file'] = [file for x in range(len(df))]
            annotations = pd.concat([annotations, df])
    annotations = annotations[['cui','name','file']]
    return annotations

def readCuiType(cuis):
    "Reads in a dictionary mapping cuis to semantic types."
    
    # Load UMLS semantic type mapping file.
    try:
        mrsty = pd.read_table('umls/mrsty.txt',sep='|',header=0,names=['cui','tui','stn','type','atui','cvf'])[['cui','tui','type']]
        mrsty = mrsty[mrsty.cui.isin(cuis)]
    except:
        raise('NOTE: Must have previously created umls/mrsty.txt by running "Load UMLS data.ipynb" to run readCuiType()')
    
    cuiType = {}
    for x in mrsty.iterrows():
        if x[1].cui not in cuiType:
            cuiType[x[1].cui] = []
        cuiType[x[1].cui].append(x[1].type)
    return cuiType

def getStats(df):
    "Gets stats for given dataframe"
    n = len(df)
    tp = sum(df.prediction == df.goldCui)
    fp = sum((df.prediction != df.goldCui) & (df.normalized==True))
    recall = round(tp/n,2) if n > 0 else 0
    precision = round(tp/(tp+fp),2) if (tp+fp) > 0 else 0
    return n, tp, fp, recall, precision

def sieveResults(results):
    "Returns a sieve-level analysis of results."
    levels = range(1,max(results.normalizingSieveLevel)+1)
    sieves = pd.DataFrame([], columns=['sieve','n','tp','fp', 'sieve_acc', 'agg_recall', 'agg_precision'])
    
    # Results for each sieve
    for i in levels:
        df = results[results.normalizingSieveLevel==i]
        n, tp, fp, recall, precision = getStats(df)
        sieve = df.normalizingSieveName.iloc[0] if n > 0 else "Unknown"
        sieves.loc[i] = [sieve, n, tp, fp, recall, 0, 0]
        sieves.loc[i,'agg_recall'] = round(sum(sieves.tp)/sum(sieves.n),2)
        sieves.loc[i,'agg_precision'] = round(sum(sieves.tp)/(sum(sieves.tp)+sum(sieves.fp)),2)
    
    # Total results
    n, tp, fp, recall, precision = getStats(results)
    sieves.loc[i+1] = ['Total', n, tp, fp, '-', recall, precision]
    return sieves

def stratifyByCol(df, col):
    "Stratifies the results by given column."    
    # If the column is a list, explode list into individual rows
    if (df.sample(100).applymap(type).mode(0).astype(str) == "<class 'list'>")[col][0]:
        df = df.explode(col)
        
    rows = []
    for key in set(df[col]):
        if not pd.isnull(key):
            sub = df[df[col]==key]
            rows.append([key] + list(getStats(sub)))
    return pd.DataFrame(rows, columns=[col,'n','tp','fp', 'recall', 'precision']).sort_values('precision',ascending=False)

def getAmbiguous(df):
    "Find ambiguous names"
    dfMap = {}
    for i in range(len(df)):
        name = df.iloc[i]['name'].lower().strip()
        cui = df.iloc[i]['cui'].lower().strip()

        if name not in dfMap:
            dfMap[name] = []

        dfMap[name] = list(set([cui] + dfMap[name]))

    namesToCuis = pd.DataFrame(dfMap.items(),columns=['name','cuis'])
    namesToCuis['ambiguous'] = [len(x) > 1 for x in namesToCuis.cuis]
    return namesToCuis[namesToCuis.ambiguous]

In [225]:
%%time
# Setup: Load terminology into dictionary, train, test, and results
dataset = 'n2c2'
cuiName, nameCui = readTerminology(f'../resources/{dataset}_terminology.txt')
train = readAnnotations(f'../{dataset}-data/train')
test = readAnnotations(f'../{dataset}-data/test')
results = pd.read_csv(f'../{dataset}-data/output/results.txt',sep='\t')

# Load semantic type map
cuis = list(set(list(results.goldCui) + list(results.prediction)))
cuiType = readCuiType(cuis)

# Create analysis dataframe
cols = ['normalized','normalizingSource','normalizingSieveName','name','prediction','goldCui','namePermutations']
analysis = results[cols]
analysis = analysis.assign(goldNames=[['CUI-less'] if c=='CUI-less' else cuiName[c] if c in cuiName else ['Missing'] for c in results.goldCui])
analysis = analysis.assign(predTypes=[cuiType[c] if c in cuiType else ['Missing'] for c in results.prediction])
analysis = analysis.assign(goldTypes=[cuiType[c] if c in cuiType else ['Missing'] for c in results.goldCui])

# Sanity checks
assert len(analysis[analysis.normalized & (analysis.predTypes=='Missing')])==0, 'Predicted CUI missing ST'
assert len(analysis[(analysis.goldCui != 'CUI-less') & analysis.goldTypes=='Missing'])==0, 'Gold CUI missing ST'
assert len(analysis[analysis.goldNames=='Missing'])==0, 'Gold names missing'
assert len(analysis[analysis.goldTypes=='Missing'])==0, 'Gold types missing'

C:\Users\etfrench\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


Wall time: 8.71 s


C:\Users\etfrench\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [256]:
# Stratify performance by column
normalized = analysis[analysis.normalized]
stratifyByCol(normalized, 'normalizingSource')
stratifyByCol(normalized, 'normalizingSieveName')
stratifyByCol(normalized, 'predTypes')
stratifyByCol(normalized, 'goldTypes')

,goldTypes,n,tp,fp,recall,precision
42,Bacterium,3,3,0,1.00,1.00
21,Injury or Poisoning,9,9,0,1.00,1.00
22,"Element, Ion, or Isotope",1,1,0,1.00,1.00
43,Individual Behavior,1,1,0,1.00,1.00
26,Neoplastic Process,13,13,0,1.00,1.00
27,Idea or Concept,2,2,0,1.00,1.00
45,Congenital Abnormality,2,2,0,1.00,1.00
30,Chemical Viewed Structurally,1,1,0,1.00,1.00
31,Mental or Behavioral Dysfunction,3,3,0,1.00,1.00
32,Social Behavior,1,1,0,1.00,1.00


In [243]:
s = '3'
print(nameCui[s],[cuiName[x] for x in nameCui[s]])
train[train.name == '3']

['C0205449'] [['3', '3 (qualifier value)', 'arabic numeral 3', 'arabic numeral 3 (qualifier value)', 'three', 'three (qualifier value)']]


,cui,name,file


In [249]:
train_names = set(train.name)
train_names
errors.name.isin(train_names)

77      False
99      False
286     False
342     False
352     False
        ...  
1988    False
2035    False
2042    False
2054    False
2058    False
Name: name, Length: 61, dtype: bool

In [250]:
remove = ['normalized','normalizingSource','normalizingSieveName']#,'goldTypes','predTypes'
errors = analysis[(results.prediction != results.goldCui) & (results.normalized==True)]
errors = errors[(errors.normalizingSource=='standardTerminology') & (errors.normalizingSieveName=='ExactMatchSieve')]
errors = errors.loc[:, ~errors.columns.isin(remove)]
errors['in_train'] = errors.name.isin(train_names)
print(len(errors))
errors.style.set_properties(subset=['goldNames'], **{'width': '1000px'})

61


,name,prediction,goldCui,namePermutations,goldNames,predTypes,goldTypes,in_train
77,sedation,C0344106,C0235195,sedation,"['[d]sedation', '[d]sedation (context-dependent category)', '[d]sedation (situation)', 'sedated', 'sedated (finding)', 'sedated state', 'under sedation']",['Therapeutic or Preventive Procedure'],['Finding'],False
99,enhancement,C1627358,C0443285,enhancement,"['radiolucent', 'radiolucent (qualifier value)']",['Therapeutic or Preventive Procedure'],['Qualitative Concept'],False
286,nebulizer,C0027524,C2919541,nebulizer,"['administration of medication using nebuliser mask', 'administration of medication using nebulizer mask', 'administration of medication using nebulizer mask (procedure)', 'nebuliser therapy using mask', 'nebulizer therapy using mask']",['Medical Device'],['Therapeutic or Preventive Procedure'],False
342,stabbing,C1455792,C0278145,stabbing,"['knifelike pain', 'stabbing pain', 'stabbing pain (finding)']",['Qualitative Concept'],['Sign or Symptom'],False
352,stabbing,C1455792,C0278145,stabbing,"['knifelike pain', 'stabbing pain', 'stabbing pain (finding)']",['Qualitative Concept'],['Sign or Symptom'],False
367,hydrocodone,C0020264,C0717367,hydrocodone,"['acetaminophen / hydrocodone', 'acetaminophen and hydrocodone product', 'acetaminophen- and hydrocodone-containing product', 'hydrocodone and paracetamol product', 'hydrocodone- and paracetamol-containing product', 'product containing hydrocodone and paracetamol', 'product containing hydrocodone and paracetamol (medicinal product)']","['Organic Chemical', 'Pharmacologic Substance']",['Pharmacologic Substance'],False
412,laboratory studies,C0681827,C0022885,laboratory studies,"['general laboratory procedure', 'general laboratory procedure (procedure)', 'general laboratory procedure -retired-', 'general laboratory procedure, nos', 'investig.- lab.,general', 'investigation - lab.,general', 'lab. test - general', 'laboratory procedure', 'laboratory procedure (procedure)', 'laboratory procedure - general - nos', 'laboratory procedure - general - nos (context-dependent category)', 'laboratory procedure - general - nos (procedure)', 'laboratory procedure - general - nos (situation)', 'laboratory procedure nos', 'laboratory procedure nos (procedure)', 'laboratory procedures', 'laboratory procedures (procedure)', 'laboratory procedures -general', 'laboratory procedures -general (context-dependent category)', 'laboratory procedures -general (situation)', 'laboratory test', 'laboratory test (procedure)', 'laboratory test, nos', 'procedure, lab.-general', 'test, lab. - general']",['Laboratory Procedure'],['Laboratory Procedure'],False
447,suture,C0038969,C0009068,suture,"['closure by suture', 'closure by suture (procedure)', 'closure by suture, nos', 'closure by suturing - action', 'closure by suturing - action (qualifier value)', 'suture, nos', 'suturing', 'suturing - action']",['Medical Device'],['Therapeutic or Preventive Procedure'],False
527,suicidal,C0438696,C0424000,suicidal,"['feeling suicidal', 'feeling suicidal (finding)', 'suicidal ideation', 'suicidal thoughts', 'suicidal thoughts (finding)']",['Mental or Behavioral Dysfunction'],['Finding'],False
531,bruising,C0009938,C0423514,bruising,"['mastoid cavity finding', 'mastoid cavity finding (finding)', 'mastoid cavity observation']",['Injury or Poisoning'],['Finding'],False


In [257]:
# results = pd.read_csv(f'../{dataset}-data/output/results.txt',sep='\t')
sieveResults(results)

,sieve,n,tp,fp,sieve_acc,agg_recall,agg_precision
1,ExactMatchSieve,1309,1221,88,0.93,0.93,0.93
2,AbbreviationExpansionSieve,14,3,11,0.21,0.93,0.93
3,PrepositionalTransformSieve,13,10,3,0.77,0.92,0.92
4,Unknown,0,0,0,0,0.92,0.92
5,HyphenationSieve,2,2,0,1,0.92,0.92
6,DiseaseTermSynonymsSieve,16,2,14,0.12,0.91,0.91
7,StemmingSieve,17,6,11,0.35,0.91,0.91
8,Total,2062,1244,127,-,0.6,0.91


In [258]:
results = pd.read_csv(f'../{dataset}-data/output/results.txt',sep='\t')
sieveResults(results)

,sieve,n,tp,fp,sieve_acc,agg_recall,agg_precision
1,ExactMatchSieve,4283,4032,251,0.94,0.94,0.94
2,AbbreviationExpansionSieve,95,38,57,0.4,0.93,0.93
3,PrepositionalTransformSieve,29,26,3,0.9,0.93,0.93
4,Unknown,0,0,0,0,0.93,0.93
5,HyphenationSieve,10,10,0,1,0.93,0.93
6,DiseaseTermSynonymsSieve,46,8,38,0.17,0.92,0.92
7,StemmingSieve,79,33,46,0.42,0.91,0.91
8,Total,6621,4147,395,-,0.63,0.91


In [262]:
len(set(results.filename))

50

In [192]:
# getAmbiguous(train)

In [11]:
# omissions = results[results.prediction.isnull() & results.goldNames.notnull()]
# omissions[['filename','name','namePermutations','goldCui','goldNames']].head(5)